# Prompt Explorer

This is a notebook for exploring prompts and seeds. Random seeds are generated and displayed. The idea is to have this running alongside other more complicated notebooks, but this one doesn't needs to have gdrive connection. Then when finding cool prompt/seed combos, but them in the prompts google sheet and have those more complex notebooks reference the sheet vs defining prompts as variables in the code. 

In [ ]:
%%capture
!pip install diffusers
!pip install transformers scipy ftfy accelerate

import os 
import pandas as pd
import numpy as np

from google.colab import auth

import gspread
from google.auth import default
from google.colab import drive
import torch
from diffusers import StableDiffusionPipeline

from PIL import Image

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid


In [ ]:
connect_google_drive = True #@param {type:"boolean"}

if connect_google_drive:

    drive.mount('/content/gdrive')

    auth.authenticate_user()

    creds, _ = default()

    gc = gspread.authorize(creds)

Mounted at /content/gdrive


In [ ]:


pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4",
                                               torch_dtype=torch.float16,
                                               safety_checker=None
                                               )  


pipe = pipe.to("cuda")

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


In [24]:

code_folder = r'/content/gdrive/MyDrive/AI Music Visuals Share/New Codes'

prompt_sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1YhC-iejPPC0bV4OJlxqZIQJj3FJqM-6xYauY7TB8EQA')

sheets = prompt_sheet.worksheet('prompts').get_all_values()
df_prompt = pd.DataFrame(sheets[1:], columns=sheets[0]).set_index('name')

df_prompt

,prompt,seeds,guidance_scale
name,,,
succulent,"a giant succulent in the desert in the sand, t...",5842234013956479,5
reflected galaxy,underground lake with a galaxy reflected in th...,5883559517894420,50
red cave1,"descending deep into a cave, red lava on the w...",4866412378884770 4940561922459094,7.5
alien dunes,rolling dunes on an alien planet at night vivi...,5453396741977211 6583682090290432 648251554957...,7.5
cornu,"standing wave pattern, rainbow, wishing well",5842234013956479,50
alien mountain,"inspired by David A. Hardy, digital landscape ...",4092123727786949 7285707432238947,7.5
overgrown fantasy,a foggy view of a bridge in a fantasy overgrow...,5618257995059877 4638125635335835 823596687782...,7.5
falling in cave,"descending deep into a cave, red lava on the w...",5451964648290255,7.5
vine cave,a foggy view in a fantasy overgrown vines worl...,7078380737840840,7.5


In [26]:
# # https://huggingface.co/docs/diffusers/using-diffusers/reusing_seeds

prompt_name = 'red cave2'
prompt = df_prompt['prompt'][prompt_name]
guidance_scale = float(df_prompt['guidance_scale'][prompt_name])

# prompt = """

# """

# rows X cols of images. Reduce for speed and memory issues. 
rows = 2
cols = 2

num_images = rows*cols

# Make new random seeds in a hacky way. TODO: probably a function to generate seeds without making a generator instance. 

generator = torch.Generator(device="cuda")
seeds = [generator.seed() for i in range(num_images)]
generator = [torch.Generator(device="cuda").manual_seed(seed) for seed in seeds]

print("Seeds: {}".format(seeds))

images = pipe(prompt, generator=generator, num_images_per_prompt=rows*cols).images

image_grid(images, rows=rows, cols=cols)

Output hidden; open in https://colab.research.google.com to view.